In [1]:
# imports

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
# Step 1 — load the two CSV files

In [4]:
df1 = pd.read_csv('rwap25_gis_dataset1.csv')   # assets
df2 = pd.read_csv('rwap25_gis_dataset2.csv')   # zip time-series

print("DF1 rows,cols:", df1.shape)
print("DF2 rows,cols:", df2.shape)
print("DF1 columns:", df1.columns.tolist())
print("DF2 columns (sample):", df2.columns[:12].tolist())

DF1 rows,cols: (8652, 18)
DF2 rows,cols: (26314, 316)
DF1 columns: ['Location Code', 'Real Property Asset Name', 'Installation Name', 'Owned or Leased', 'GSA Region', 'Street Address', 'City', 'State', 'Zip Code', 'Latitude', 'Longitude', 'Building Rentable Square Feet', 'Available Square Feet', 'Construction Date', 'Congressional District', 'Congressional District Representative Name', 'Building Status', 'Real Property Asset Type']
DF2 columns (sample): ['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName', '31-01-2000', '29-02-2000', '31-03-2000']


In [5]:
# Step 2 — keep only zip-level rows from Dataset 2

In [6]:
# If RegionType exists, filter rows where it's 'zip' (case-insensitive).
if 'RegionType' in df2.columns:
    df2_zip = df2[df2['RegionType'].astype(str).str.lower() == 'zip'].copy()
else:
    df2_zip = df2.copy()

print("Filtered df2 to zip rows:", df2_zip.shape)

Filtered df2 to zip rows: (26314, 316)


In [7]:
# Step 3 — standardize ZIP strings in both dataframes

In [8]:
# For df1: create a standardized 'zip' column from 'Zip Code'
if 'Zip Code' in df1.columns:
    df1['Zip_raw'] = df1['Zip Code'].astype(str).str.strip().fillna('')
else:
    # fallback: take first column name that contains 'zip'
    zip_col = [c for c in df1.columns if 'zip' in c.lower()]
    df1['Zip_raw'] = df1[zip_col[0]].astype(str).str.strip() if zip_col else ''

df1['zip'] = df1['Zip_raw'].str.extract(r'(\d+)', expand=False).fillna('').str.zfill(5)

# For df2_zip: extract numeric from RegionName -> zip
if 'RegionName' in df2_zip.columns:
    df2_zip['RegionName_raw'] = df2_zip['RegionName'].astype(str).str.strip().fillna('')
    df2_zip['zip'] = df2_zip['RegionName_raw'].str.extract(r'(\d+)', expand=False).fillna('').str.zfill(5)
else:
    # fallback: find any column that looks like RegionName
    cand = [c for c in df2_zip.columns if 'region' in c.lower() or 'zip' in c.lower()]
    if cand:
        df2_zip['RegionName_raw'] = df2_zip[cand[0]].astype(str).str.strip().fillna('')
        df2_zip['zip'] = df2_zip['RegionName_raw'].str.extract(r'(\d+)', expand=False).fillna('').str.zfill(5)
    else:
        df2_zip['RegionName_raw'] = ''
        df2_zip['zip'] = ''

print("Unique zips in df1:", df1['zip'].nunique())
print("Unique zips in df2_zip:", df2_zip['zip'].nunique())

Unique zips in df1: 3581
Unique zips in df2_zip: 26314


In [9]:
# Step 4 — find the date columns in Dataset 2 and melt it to long format

In [10]:
# find columns matching dd-mm-yyyy (e.g. '31-07-2025')
date_pattern = re.compile(r'^\d{2}-\d{2}-\d{4}$')
date_cols = [c for c in df2_zip.columns if date_pattern.match(c)]

print("Found date columns count:", len(date_cols))
print("Sample date columns:", date_cols[:6])

# melt wide->long: id_vars are everything except date columns
id_vars = [c for c in df2_zip.columns if c not in date_cols]
df2_long = df2_zip.melt(id_vars=id_vars, value_vars=date_cols,
                        var_name='date_str', value_name='price_raw')

# parse date and numeric price
df2_long['date'] = pd.to_datetime(df2_long['date_str'], format='%d-%m-%Y', errors='coerce')
df2_long['price'] = pd.to_numeric(df2_long['price_raw'], errors='coerce')

# drop invalid date rows (if any)
df2_long = df2_long.dropna(subset=['date']).copy()

print("df2_long rows:", df2_long.shape)

Found date columns count: 307
Sample date columns: ['31-01-2000', '29-02-2000', '31-03-2000', '30-04-2000', '31-05-2000', '30-06-2000']
df2_long rows: (8078398, 15)


In [11]:
# Step 5 — compute the latest non-null price and its date per ZIP

In [12]:
# keep only rows where price is not null, then take the last (latest date) per zip
df2_long_nonnull = df2_long.dropna(subset=['price']).copy()

# if there are no non-null prices we'll get an empty df
if df2_long_nonnull.empty:
    print("Warning: dataset2 has no non-null price values.")
    latest_per_zip = pd.DataFrame(columns=['zip','price_latest','price_latest_date'])
else:
    latest_per_zip = (df2_long_nonnull.sort_values(['zip','date'])
                                  .groupby('zip', as_index=False)
                                  .last()[['zip','price','date']])
    latest_per_zip = latest_per_zip.rename(columns={'price':'price_latest','date':'price_latest_date'})

print("Zips with latest price available:", len(latest_per_zip))
latest_per_zip.head()

Zips with latest price available: 26314


,zip,price_latest,price_latest_date
0,01001,340459.1165,2025-07-31
1,01002,538321.8056,2025-07-31
2,01005,409267.9918,2025-07-31
3,01007,467503.3545,2025-07-31
4,01008,372725.8783,2025-07-31


In [13]:
# Step 6 — merge latest-per-zip info back with meta (City/State) and attach to Dataset1

In [15]:
# pick some metadata from df2_zip (if present)
meta_cols = []
for c in ['RegionID','City','State','CountyName']:
    if c in df2_zip.columns:
        meta_cols.append(c)
meta_cols = list(dict.fromkeys(meta_cols))  # keep order & unique

zip_meta = df2_zip[meta_cols + ['zip']].drop_duplicates(subset=['zip'], keep='first')

# create zip_info: meta + latest price
zip_info = zip_meta.merge(latest_per_zip, on='zip', how='left')

# left-merge into df1 (many assets per zip -> one zip_info row)
merged = df1.merge(zip_info, on='zip', how='left', validate='m:1')

# initial source flag: exact_zip if we have a price_latest
# Ensure the 'source' column is of type object to handle mixed types
merged['source'] = np.where(merged['price_latest'].notna(), 'exact_zip', None)

print("Merged rows:", merged.shape)
print("Fraction with price_latest:", merged['price_latest'].notna().mean())

Merged rows: (8652, 27)
Fraction with price_latest: 0.9288025889967637


In [16]:
# Step 7 — inspect assets that don’t have price_latest

In [17]:
missing = merged[merged['price_latest'].isna()].copy()
print("Number of assets missing price_latest:", len(missing))
# show unique missing zips and their df2_long summary
missing_zips = missing['zip'].unique().tolist()[:50]
print("Sample missing zips (up to 50):", missing_zips)

# For diagnostics: show df2_long info for those zips
diag = (df2_long[df2_long['zip'].isin(missing_zips)]
        .groupby('zip').agg(n_obs=('price','size'),
                            n_non_null=('price', lambda s: s.notna().sum()),
                            first_date=('date','min'),
                            last_date=('date','max')).reset_index())
print(diag.head(30))

Number of assets missing price_latest: 616
Sample missing zips (up to 50): ['20993', '47907', '85620', '80225', '00968', '70803', '59482', '78567', '20192', '59256', '88029', '83853', '24011', '96799', '99752', '59542', '92283', '05460', '76155', '08608', '97204', '59411', '04491', '10278', '52801', '20373', '99780', '58329', '00716', '00708', '20585', '00820', '04936', '00641', '20503', '00784', '85633', '10038', '79711', '14604', '77010', '63145', '87026', '20201', '79839', '47405', '96950', '00918', '78235', '00824']
Empty DataFrame
Columns: [zip, n_obs, n_non_null, first_date, last_date]
Index: []


In [ ]:
# Step 8 — save the merged file (with source) so you have a checkpoint

In [18]:
merged.to_csv('dataset1_merged_step.csv', index=False)
print("Saved merged file to dataset1_merged_step.csv")

Saved merged file to dataset1_merged_step.csv


In [ ]:
# STEP A — quick read/setup (if not done already)

In [19]:
import pandas as pd, numpy as np, re
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from math import radians, cos, sin, asin, sqrt

merged = pd.read_csv('dataset1_merged_step.csv')  # file from earlier step

In [ ]:
# for columns:

In [ ]:
# Quick inspect (see what's filled)

In [20]:
# show counts and a sample
print("Non-null counts:")
print("State_x:", merged['State_x'].notna().sum(), "/", len(merged))
print("State_y:", merged['State_y'].notna().sum(), "/", len(merged))

print("\nSample pairs (first 20 rows):")
print(merged[['State_x','State_y']].head(20))

Non-null counts:
State_x: 8652 / 8652
State_y: 8036 / 8652

Sample pairs (first 20 rows):
   State_x State_y
0       GA      GA
1       WI      WI
2       MN      MN
3       MD      MD
4       CO      CO
5       CO      CO
6       FL      FL
7       AZ      AZ
8       FL      FL
9       MD     NaN
10      CA      CA
11      CA      CA
12      IN     NaN
13      AZ      AZ
14      PA      PA
15      UT      UT
16      FL      FL
17      MD      MD
18      PA      PA
19      AZ      AZ


In [ ]:
# Block 1 — Create unified State and state_source

In [21]:
# run this first (assumes merged DataFrame exists)
import pandas as pd, numpy as np

# prefer State_x, else State_y
merged['State_unified_raw'] = merged['State_x'].where(
    merged['State_x'].notna() & (merged['State_x'].astype(str).str.strip()!=''),
    merged['State_y']
)

# normalize whitespace and uppercase, convert empty/'nan' strings to None
merged['State_unified_raw'] = merged['State_unified_raw'].astype(str).str.strip().replace({'': None, 'nan': None, 'None': None})
merged['State_unified_raw'] = merged['State_unified_raw'].where(merged['State_unified_raw'].notna(), None)
merged['State_unified_raw'] = merged['State_unified_raw'].apply(lambda x: x.upper() if pd.notna(x) else x)

# simple full-name -> abbrev mapping (extend if needed)
state_map = {
    'ALABAMA':'AL','ALASKA':'AK','ARIZONA':'AZ','ARKANSAS':'AR','CALIFORNIA':'CA',
    'COLORADO':'CO','CONNECTICUT':'CT','DELAWARE':'DE','FLORIDA':'FL','GEORGIA':'GA',
    'HAWAII':'HI','IDAHO':'ID','ILLINOIS':'IL','INDIANA':'IN','IOWA':'IA','KANSAS':'KS',
    'KENTUCKY':'KY','LOUISIANA':'LA','MAINE':'ME','MARYLAND':'MD','MASSACHUSETTS':'MA',
    'MICHIGAN':'MI','MINNESOTA':'MN','MISSISSIPPI':'MS','MISSOURI':'MO','MONTANA':'MT',
    'NEBRASKA':'NE','NEVADA':'NV','NEW HAMPSHIRE':'NH','NEW JERSEY':'NJ','NEW MEXICO':'NM',
    'NEW YORK':'NY','NORTH CAROLINA':'NC','NORTH DAKOTA':'ND','OHIO':'OH','OKLAHOMA':'OK',
    'OREGON':'OR','PENNSYLVANIA':'PA','RHODE ISLAND':'RI','SOUTH CAROLINA':'SC',
    'SOUTH DAKOTA':'SD','TENNESSEE':'TN','TEXAS':'TX','UTAH':'UT','VERMONT':'VT',
    'VIRGINIA':'VA','WASHINGTON':'WA','WEST VIRGINIA':'WV','WISCONSIN':'WI','WYOMING':'WY',
    'DISTRICT OF COLUMBIA':'DC', 'DC':'DC'
}

def to_state_abbrev(s):
    if s is None: return None
    s_up = str(s).strip().upper()
    if len(s_up) == 2 and s_up.isalpha(): 
        return s_up
    return state_map.get(s_up, s_up)  # fallback keep as-is for manual inspection

merged['State'] = merged['State_unified_raw'].apply(to_state_abbrev)

# mark source
def state_source(row):
    sx = row.get('State_x')
    sy = row.get('State_y')
    if pd.notna(sx) and str(sx).strip()!='' and pd.notna(sy) and str(sy).strip()!='':
        if str(sx).strip().upper() == str(sy).strip().upper():
            return 'both_same'
        else:
            return 'both_conflict'
    if pd.notna(sx) and str(sx).strip()!='':
        return 'state_x'
    if pd.notna(sy) and str(sy).strip()!='':
        return 'state_y'
    return 'none'

merged['state_source'] = merged.apply(state_source, axis=1)

# Quick check
print("Unified State non-null:", merged['State'].notna().sum(), "/", len(merged))
print("state_source counts:\n", merged['state_source'].value_counts())
print("Sample unified states:", merged['State'].unique()[:20])

Unified State non-null: 8652 / 8652
state_source counts:
 state_source
both_same        8035
state_x           616
both_conflict       1
Name: count, dtype: int64
Sample unified states: ['GA' 'WI' 'MN' 'MD' 'CO' 'FL' 'AZ' 'CA' 'IN' 'PA' 'UT' 'TX' 'MO' 'NM'
 'OR' 'VA' 'WY' 'LA' 'KY' 'MI']


In [ ]:
# Inspect the one conflict and resolve it (quick)

In [22]:
# show conflicting rows where both exist but are different
conflicts = merged[(merged['State_x'].notna()) & (merged['State_y'].notna()) &
                   (merged['State_x'].astype(str).str.strip().str.upper() != merged['State_y'].astype(str).str.strip().str.upper())]

print("Number of conflicts:", len(conflicts))
conflicts[['Location Code','State_x','State_y','City_x','City_y','zip','Latitude','Longitude']].head(20)

Number of conflicts: 1


,Location Code,State_x,State_y,City_x,City_y,zip,Latitude,Longitude
1497,NJ0148,NJ,NY,NEWARK,New Windsor,12553,40.728366,-74.174679


In [ ]:
# solving the conflict - 

In [ ]:
# 1) Install & import (only if needed)

In [23]:
import pgeocode
import pandas as pd

ModuleNotFoundError: No module named 'pgeocode'

In [ ]:
# Use ZIP-to-state lookup to get authoritative state code

In [ ]:
# create pgeocode nominatim object for US
nomi = pgeocode.Nominatim('us')

# function to lookup 2-letter state by zip (returns None if not found)
def zip_to_state_abbrev(zipcode):
    try:
        info = nomi.query_postal_code(str(zipcode).zfill(5))
        # pgeocode returns NaN for missing fields; state_code property is abbreviation if present
        st = info['state_code']
        if pd.isna(st):
            return None
        return str(st).strip().upper()
    except Exception as e:
        return None

# test on the conflict zip (12553)
print("ZIP 12553 ->", zip_to_state_abbrev('12553'))  # expected 'NY'

ZIP 12553 -> NY


In [ ]:
# Resolve all conflicts using the ZIP lookup and record provenance

In [ ]:
# Apply zip lookup for all conflicted rows and update the merged DataFrame
for idx, row in conflicts.iterrows():
    z = row['zip']
    authoritative_state = zip_to_state_abbrev(z)
    if authoritative_state:
        # keep original values for audit
        merged.at[idx, 'State_before_resolution'] = merged.at[idx, 'State']  # current unified state
        merged.at[idx, 'State_resolved_by_zip'] = authoritative_state
        # update unified State to authoritative zip result
        merged.at[idx, 'State'] = authoritative_state
        merged.at[idx, 'state_source'] = 'zip_lookup_override'
    else:
        # if the lookup failed, leave current State as-is but flag for manual review
        merged.at[idx, 'State_resolve_note'] = 'zip_lookup_failed_manual_review'
        merged.at[idx, 'state_source'] = 'conflict_needs_manual'

In [ ]:
# Audit the change (show rows that were modified)

In [ ]:
# show those rows you just modified
resolved = merged[merged['state_source']=='zip_lookup_override']
print("Resolved rows via ZIP lookup:", len(resolved))
print(resolved[['Location Code','zip','State_x','State_y','State_before_resolution','State_resolved_by_zip','State','state_source']])

Resolved rows via ZIP lookup: 1
     Location Code    zip State_x State_y State_before_resolution  \
1497        NJ0148  12553      NJ      NY                      NJ   

     State_resolved_by_zip State         state_source  
1497                    NY    NY  zip_lookup_override  


In [ ]:
# Save an audit log

In [ ]:
# Save the conflict-resolution audit for traceability
audit = merged.loc[conflicts.index, ['Location Code','zip','State_x','State_y','State_before_resolution','State_resolved_by_zip','State','state_source']]
audit.to_csv('state_resolution_audit.csv', index=False)
print("Saved state resolution audit to state_resolution_audit.csv")

Saved state resolution audit to state_resolution_audit.csv


In [ ]:
# again

In [ ]:
import pandas as pd

# Load the merged dataset (replace with your merged CSV file name)
df = pd.read_csv("dataset1_merged_step.csv")

print("Initial shape:", df.shape)
print("Columns available:", df.columns)


Initial shape: (8652, 27)
Columns available: Index(['Location Code', 'Real Property Asset Name', 'Installation Name',
       'Owned or Leased', 'GSA Region', 'Street Address', 'City_x', 'State_x',
       'Zip Code', 'Latitude', 'Longitude', 'Building Rentable Square Feet',
       'Available Square Feet', 'Construction Date', 'Congressional District',
       'Congressional District Representative Name', 'Building Status',
       'Real Property Asset Type', 'Zip_raw', 'zip', 'RegionID', 'City_y',
       'State_y', 'CountyName', 'price_latest', 'price_latest_date', 'source'],
      dtype='object')


In [ ]:
def resolve_state(row):
    # If both match → use either
    if pd.notna(row['State_x']) and pd.notna(row['State_y']):
        if row['State_x'] == row['State_y']:
            return row['State_x'], 'both_same'
        else:
            return row['State_x'], 'both_conflict'
    # If only State_x exists
    elif pd.notna(row['State_x']):
        return row['State_x'], 'state_x'
    # If only State_y exists
    elif pd.notna(row['State_y']):
        return row['State_y'], 'state_y'
    # If both are null
    else:
        return None, 'missing'

df[['Unified_State', 'state_source']] = df.apply(resolve_state, axis=1, result_type="expand")


In [ ]:
conflicts = df[df['state_source'] == 'both_conflict']
print(f"Number of conflicts: {len(conflicts)}")

if len(conflicts) > 0:
    print(conflicts[['Location Code','State_x','State_y','City_x','City_y','zip']])


Number of conflicts: 1
     Location Code State_x State_y  City_x       City_y    zip
1497        NJ0148      NJ      NY  NEWARK  New Windsor  12553


In [ ]:
# For NJ0148, correct Unified_State to NJ
df.loc[df['Location Code'] == 'NJ0148', 'Unified_State'] = 'NJ'
df.loc[df['Location Code'] == 'NJ0148', 'state_source'] = 'resolved_conflict'


In [ ]:
df.drop(columns=['State_x', 'State_y'], inplace=True)


In [ ]:
df.to_csv("state_resolution_audit.csv", index=False)
print("✅ Clean dataset saved as: state_resolution_audit.csv")


✅ Clean dataset saved as: state_resolution_audit.csv


In [ ]:
# unifying the cities

In [ ]:
# Step 1 — Load the cleaned dataset

In [ ]:
# Load the cleaned dataset from the previous step
df = pd.read_csv("state_resolution_audit.csv")

print("Shape:", df.shape)
print("Columns:", df.columns)

Shape: (8652, 27)
Columns: Index(['Location Code', 'Real Property Asset Name', 'Installation Name',
       'Owned or Leased', 'GSA Region', 'Street Address', 'City_x', 'Zip Code',
       'Latitude', 'Longitude', 'Building Rentable Square Feet',
       'Available Square Feet', 'Construction Date', 'Congressional District',
       'Congressional District Representative Name', 'Building Status',
       'Real Property Asset Type', 'Zip_raw', 'zip', 'RegionID', 'City_y',
       'CountyName', 'price_latest', 'price_latest_date', 'source',
       'Unified_State', 'state_source'],
      dtype='object')


In [ ]:
# Step 2 — Create Unified_City column

In [ ]:
# We’ll define rules for merging City_x and City_y.

In [ ]:
def resolve_city(row):
    city_x = str(row['City_x']).strip() if pd.notna(row['City_x']) else None
    city_y = str(row['City_y']).strip() if pd.notna(row['City_y']) else None

    # If both are present and same → use either
    if city_x and city_y:
        if city_x.lower() == city_y.lower():  # case-insensitive match
            return city_x, 'both_same'
        else:
            return city_x, 'both_conflict'  # temporarily mark conflict

    # If only city_x exists
    elif city_x:
        return city_x, 'city_x'

    # If only city_y exists
    elif city_y:
        return city_y, 'city_y'

    # If both are missing
    else:
        return None, 'missing'

# Apply function to dataframe
df[['Unified_City', 'city_source']] = df.apply(resolve_city, axis=1, result_type="expand")

In [ ]:
# Step 3 — Check conflicts

In [ ]:
conflicts = df[df['city_source'] == 'both_conflict']
print(f"Number of city conflicts: {len(conflicts)}")

# Show a sample of mismatched cities
print(conflicts[['Location Code', 'City_x', 'City_y', 'zip']].head(20))

Number of city conflicts: 660
    Location Code                 City_x              City_y    zip
26         NM0576               PLACITAS        Santa Teresa  88008
48         MI2137               MUSKEGON      Roosevelt Park  49441
63         NY7402               BROOKLYN            New York  11201
75         AR0062     HOT SPGS NATL PARK         Hot Springs  71901
87         IL2520             BELLEVILLE              Shiloh  62221
91         NC2314        WASHINGTON PARK          Washington  27889
98         IL2491            SPRINGFIELD        Leland Grove  62704
101        MO0617              ST. LOUIS         Saint Louis  63120
128        MO0610              ST. LOUIS         Saint Louis  63120
157        GA1158                ATLANTA            Chamblee  30341
168        MI2187  CHESTERFIELD TOWNSHIP        Chesterfield  48051
175        FL3183                  MIAMI       The Crossings  33186
184        OH2418              BEACHWOOD      Shaker Heights  44122
186        NJ5116 

In [ ]:
# most of your city conflicts aren’t really “true” conflicts but spelling/style/abbreviation differences or neighborhood vs. city names.

# We'll handle these smartly using the following stepwise approach:

In [ ]:
# Step 1 — Use ZIP Codes to Auto-Resolve Cities

In [ ]:
# Load your dataset
df = pd.read_csv("state_resolution_audit.csv")

# If city_y is missing, first fill it with city_x
df["City_y"] = df["City_y"].fillna(df["City_x"])

# Create a ZIP → most common city mapping
zip_city_map = (
    df.groupby("zip")["City_y"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])
    .to_dict()
)

# Apply this mapping to create a unified city column
df["Unified_City"] = df["zip"].map(zip_city_map)

# Check how many are still conflicting
df["city_conflict"] = (df["City_x"].str.upper() != df["Unified_City"].str.upper())
print("Remaining conflicts after ZIP-based resolution:", df["city_conflict"].sum())

Remaining conflicts after ZIP-based resolution: 688


In [ ]:
# Step 2 — Fuzzy Matching for Close Spellings

In [ ]:
from fuzzywuzzy import fuzz

def pick_best_city(row):
    city_x = str(row["City_x"]).upper()
    unified = str(row["Unified_City"]).upper()
    score = fuzz.ratio(city_x, unified)
    return unified if score >= 85 else city_x

df["Final_City"] = df.apply(pick_best_city, axis=1)

C:\Users\gupta\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# Step 3 — Save Final Cleaned Dataset

In [ ]:
df.drop(columns=["city_conflict"], inplace=True, errors="ignore")
df.to_csv("city_resolution_audit.csv", index=False)
print("✅ City conflicts resolved and saved to city_resolution_audit.csv")

✅ City conflicts resolved and saved to city_resolution_audit.csv


In [ ]:
# Step 4 — Recheck Conflicts

In [ ]:
conflicts = df[df["City_x"].str.upper() != df["Final_City"].str.upper()]
print("Final remaining conflicts:", len(conflicts))
print(conflicts[["Location Code", "City_x", "City_y", "Final_City", "zip"]].head(38))

Final remaining conflicts: 38
     Location Code              City_x                City_y  \
348         NC2615       WINSTON SALEM         Winston-Salem   
384         NC0113       WINSTON SALEM         Winston-Salem   
462         OH2481        BEAVER CREEK           Beavercreek   
1017        AR0066  HELENA-WEST HELENA  Helena - West Helena   
1716        HI8562              LIHU'E                 Lihue   
1886        MI3087    ST. CLAIR SHORES    Saint Clair Shores   
2066        HI7553              LIHU'E                 Lihue   
2160        MI3033     SAULT STE MARIE    Sault Sainte Marie   
2216        SC1374         MT PLEASANT        Mount Pleasant   
2244        NC1383       WINSTON SALEM         Winston-Salem   
2304        HI7793              KEA'AU                 Keaau   
2535        PA0627        WILKES BARRE          Wilkes-Barre   
2588        PA0921        WILKES BARRE          Wilkes-Barre   
2716        OH1816     ST. CLAIRSVILLE     Saint Clairsville   
2995      

In [ ]:
# resolveddddd

In [ ]:
# Step 1 — Load the cleaned merged dataset

In [ ]:
# Load the dataset after state conflict resolution
df = pd.read_csv("conflict_resolved.csv")

# Check missing price_latest values
missing_price = df['price_latest'].isna().sum()
print(f"Missing price_latest values: {missing_price}")

Missing price_latest values: 616


In [ ]:
# Step 2 — Check how missing price data is distributed

In [ ]:
# Count missing values by state
missing_by_state = df[df['price_latest'].isna()].groupby('Unified_State').size().sort_values(ascending=False)
print("Missing price counts by state:\n", missing_by_state)

# Count missing values by GSA region
missing_by_region = df[df['price_latest'].isna()].groupby('GSA Region').size().sort_values(ascending=False)
print("\nMissing price counts by GSA region:\n", missing_by_region)

KeyError: 'Unified_State'

In [ ]:
# Step 3 — Fill missing prices intelligently

In [ ]:
# Create a mapping of median prices by zip
zip_price_map = df.groupby("zip")["price_latest"].median()

# Fill missing prices using the same zip median price
df["price_latest"] = df.apply(
    lambda x: zip_price_map[x["zip"]] if pd.isna(x["price_latest"]) and x["zip"] in zip_price_map else x["price_latest"],
    axis=1
)